In [15]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [16]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [17]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [18]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [19]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [20]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [21]:
df['loc']

1     (800 South Pointe Dr APT 1401, Miami Beach, FL...
2     (17749 Collins Ave, Sunny Isles Beach, FL 3316...
3     (7046 Fisher Island Dr #7046, Miami Beach, FL ...
4     (19575 Collins Ave UNIT 38, Sunny Isles Beach,...
5     (350 Ocean Dr #403n, Key Biscayne, FL 33149, U...
6     (7951 Fisher Island Dr #7951, Miami Beach, FL ...
7     (18975 Collins Ave #1500, Sunny Isles Beach, F...
8     (8701 Collins Ave #706, Miami Beach, FL 33154,...
9     (400 Alton Rd APT 3103, Miami Beach, FL 33139,...
10    (2201 Collins Ave #801, Miami Beach, FL 33139,...
Name: loc, dtype: object

In [22]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [23]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Apogee Condo\r800 S Pointe Dr 1401\rMiami Beach,1/26/2021,5/16/2022,360,"$12,250,000.00",6595,"$2,948.96",Billy Hernandez,Douglas Elliman,Clemente Sanchez PA,Luxe Living Realty,Apogee Condo 800 S Pointe Dr 1401 Miami Beach,Apogee Condo,800 S Pointe Dr 1401 Miami Beach,"(800 South Pointe Dr APT 1401, Miami Beach, FL...","(25.7677245, -80.13634490000001, 0.0)",25.767724,-80.136345,0.0
2,Mansions at Acqualina\r17749 Collins Ave 2401/...,12/25/2021,5/18/2022,99,"$12,210,000.00",9218,"$1,324.58",Boris Vertsberger,"FIP Realty Services, LLC",Chad Carroll,"Compass Florida, LLC",Mansions at Acqualina 17749 Collins Ave 2401/2...,Mansions at Acqualina,17749 Collins Ave 2401/2402 Sunny Isles Beach,"(17749 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9400648, -80.12026379999999, 0.0)",25.940065,-80.120264,0.0
3,Fisher Island\r7046 Fisher Island Dr 7046\rMia...,5/28/2021,5/17/2022,266,"$10,400,000.00",3835,"$2,711.86",Karla Abaunza,Luxury Living Realty,Jill Eber,Coldwell Banker,Fisher Island 7046 Fisher Island Dr 7046 Miami...,Fisher Island,7046 Fisher Island Dr 7046 Miami Beach,"(7046 Fisher Island Dr #7046, Miami Beach, FL ...","(25.7623526, -80.1473326, 0.0)",25.762353,-80.147333,0.0
4,Regalia\r19575 Collins Ave 38\rSunny Isles Beach,2/4/2022,5/17/2022,35,"$10,150,000.00",7600,"$1,840.44",Chad Carroll,"Compass Florida, LLC",Roberto Morales,"Edmond Roberts, LLC",Regalia 19575 Collins Ave 38 Sunny Isles Beach,Regalia,19575 Collins Ave 38 Sunny Isles Beach,"(19575 Collins Ave UNIT 38, Sunny Isles Beach,...","(25.956929, -80.1193868, 0.0)",25.956929,-80.119387,0.0
5,Oceana Key Biscayne\r350 Ocean Dr 403N\rKey Bi...,3/9/2022,5/16/2022,12,"$7,400,000.00",3005,"$2,462.56",Monica Slodarz,"Compass Florida, LLC",Lisa Sayfie,"Compass Florida, LLC",Oceana Key Biscayne 350 Ocean Dr 403N Key Bisc...,Oceana Key Biscayne,350 Ocean Dr 403N Key Biscayne,"(350 Ocean Dr #403n, Key Biscayne, FL 33149, U...","(25.694538, -80.15759829999999, 0.0)",25.694538,-80.157598,0.0
6,Fisher Island\r7951 Fisher Island Dr 7951\rMia...,3/3/2022,5/16/2022,15,"$5,999,000.00",3140,"$1,910.51",Lazaro Rivero,"Rivero Real Estate, Inc",Karla Abaunza,Luxury Living Realty,Fisher Island 7951 Fisher Island Dr 7951 Miami...,Fisher Island,7951 Fisher Island Dr 7951 Miami Beach,"(7951 Fisher Island Dr #7951, Miami Beach, FL ...","(25.7589149, -80.13751099999999, 0.0)",25.758915,-80.137511,0.0
7,Residences by Armani Casa\r18975 Collins Ave 1...,10/19/2020,5/18/2022,540,"$5,200,000.00",3748,"$1,387.41",Melissa Barragan,Dezer Platinum Realty LLC,Neal Klotsman,Douglas Elliman,Residences by Armani Casa 18975 Collins Ave 15...,Residences by Armani Casa,18975 Collins Ave 1500 Sunny Isles Beach,"(18975 Collins Ave #1500, Sunny Isles Beach, F...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0
8,Eighty Seven Park\r8701 Collins Ave 706\rMiami...,6/2/2021,5/18/2022,327,"$4,875,000.00",3206,"$2,749.58",Diana Garchitorena,Douglas Elliman,Diana Garchitorena,Douglas Elliman,Eighty Seven Park 8701 Collins Ave 706 Miami B...,Eighty Seven Park,8701 Collins Ave 706 Miami Beach,"(8701 Collins Ave #706, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
9,Murano Grande Portofino\r400 Alton Rd 3103\rMi...,1/19/2022,5/17/2022,26,"$4,400,000.00",3058,"$1,438.85",Daniel Hertzberg,Coldwell Banker,Oliver Lloyd,Douglas Elliman,Murano Grande Portofino 400 Alton Rd 3103 Miam...,Murano Grande Portofino,400 Alton Rd 3103 Miami Beach,"(400 Alton Rd APT 3103, Miami Beach, FL 33139,...","(25.7727159, -80.14041569999999, 0.0)",25.772716,-80.140416,0.0
10,W0 South Beach\r2201 Collins Ave 801\rMiami B...,1/21/2022,5/16/2022,83,"$3,650,000.00",1543,"$2,365.52",Matthew Crane,"Compass Florida, LLC",Christopher Adeleke,Douglas Elliman,W0 South Beach 2201 Collins Ave 801 Miami B

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [24]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [35]:
df.at[10,'address_only']=('2201 Collins Ave 801')
df.at[10,'building_name']=('W South Beach')

df.at[1,'building_name']=('Apogee')
df.at[7,'building_name']=('Armani/Casa')

In [26]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [36]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 15th - May 21th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [37]:
m.save('index.html')